In [149]:
import pandas as pd
import re
import tqdm as notebook_tqdm
from transformers import AutoModel, AutoTokenizer
from razdel import tokenize
from string import punctuation
import torch

punctuation += '—'
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
vectorizer = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")
with open('../DATA/less_semantic_words.txt', 'r', encoding='utf-8') as f:
    stop_words = pd.Series(f.read().split('\n')).unique()

In [133]:
stop_words

array(['c', 'а', 'алло', ..., 'является', 'ярких', 'ясное'], dtype=object)

In [47]:
special_regex = re.compile('\d{2}.\d{2}.\d{2}')
emoj = re.compile('['
                  u'\U0001F600-\U0001F64F'
                  u'\U0001F300-\U0001F5FF'
                  u'\U0001F680-\U0001F6FF'
                  u'\U0001F1E0-\U0001F1FF'
                  u'\U00002500-\U00002BEF'
                  u'\U00002702-\U000027B0'
                  u'\U000024C2-\U0001F251'
                  u'\U0001f926-\U0001f937'
                  u'\U00010000-\U0010ffff'
                  u'\u2640-\u2642'
                  u'\u2600-\u2B55'
                  u'\u200d'
                  u'\u23cf'
                  u'\u23e9'
                  u'\u231a'
                  u'\ufe0f'
                  u'\u3030'
                  ']', re.UNICODE)

In [4]:
data = pd.read_excel('../DATA/Профессии_ОКСО_ ЛЦТ_Профилум.xlsx', index_col='№')
data.head(3)

,Отрасль текстом,Название профессии,"Специальности ОКСО - по одной специальности может быть множетсво программ обучения, которые открывают колледжии и вузы. Специальность - то, что написано у тебя в дипломе после окончания одной из программ обучения.","Уровень образования. Цифры в середине кода ОКСО хх.ХХ.хх говорят об образовательном уровне: 03 и 05 - высшее образование, 01 и 02 - среднее профессиональное образование",Описание профессии для 8-9 класса
№,,,,,
1,Агропромышленность,Агроном,"35.02.05 - Агрономия, 35.03.03 - Агрохимия и а...","SecondaryEducation,HeigherEducation,HeigherEdu...","Выращивает ягоды, овощи, фрукты, грибы, кормов..."
2,"Архитектура, строительство и недвижимость",Маркшейдер,"05.03.01 - Геология, 21.05.04 - Горное дело, 2...","HeigherEducation,HeigherEducation,HeigherEduca...","Исследует месторождения полезных ископаемых, п..."
3,"Архитектура, строительство и недвижимость",Реставратор,"07.02.01 - Архитектура, 54.02.04 - Реставрация...","SecondaryEducation,SecondaryEducation,Secondar...","Восстанавливает старинные здания, сооружения и..."


In [121]:
data.shape

(50, 5)

# Split iner data

In [101]:
def spec_symbols_remove_processor(text: str):
    text = re.sub(emoj, '', text)
    text = text.replace('\n', '')
    return text
    

def specials_processor(x):
    v = dict()
    for i, j in zip(re.findall(special_regex, x), re.split(special_regex, x)[1:]):
        v[i] = j.replace(' - ', '')
        v[i] = v[i][:-2] if v[i][-2] == ',' else v[i]
    return v


def data_education_lvl_processor(x):
    return x.split(',')


def dat_description_processor(x):
    x = spec_symbols_remove_processor(x)
    s = x.split('<br><br>')
    v = dict()
    v['main_task'] = s[0]
    v['sub_tasks'] = s[1].split('<br>')
    v['description'] = s[2].split('<br>')[1]
    return v


print(specials_processor('05.03.01 - Геология, 21.05.04 - Горное дело, 21.05.02 - Прикладная геология, 21.05.03 - Технология геологической разведки'))
print(data_education_lvl_processor('HeigherEducation,HeigherEducation,HeigherEducation,HeigherEducation'))
print(data['Описание профессии для 8-9 класса'].apply(dat_description_processor)[1])

{'05.03.01': 'Геология', '21.05.04': 'Горное дело', '21.05.02': 'Прикладная геология', '21.05.03': 'Технология геологической разведки'}
['HeigherEducation', 'HeigherEducation', 'HeigherEducation', 'HeigherEducation']
{'main_task': 'Выращивает ягоды, овощи, фрукты, грибы, кормовые травы и злаковые растения', 'sub_tasks': [' Проверяет почву и оценивает качество семян.', ' Выбирает сорт и культуру для посева.', ' Принимает решение, когда сажать растения и убирать урожай.', ' Организует работу комбайнеров и трактористов: дает им задания и контролирует их выполнение.', ' Увеличивает объем сбора урожая и повышает его качество.'], 'description': 'Представь, что агроном работает на ферме, которая выращивает кукурузу. Специалист обследует почву, изучает погоду и проводит исследования, чтобы узнать, какие удобрения подойдут для кукурузы. Затем он рассказывает фермеру, как и когда применять удобрения. Агроном также проверяет растения на наличие болезней и вредителей. Если он замечает проблемы, то

In [102]:
data_preprocessed = data.copy(deep=1)
data_preprocessed['Специальности ОКСО'] = data['Специальности ОКСО - по одной специальности может быть множетсво программ обучения, которые открывают колледжии и вузы. Специальность - то, что написано у тебя в дипломе после окончания одной из программ обучения.'].apply(specials_processor)
data_preprocessed['Уровень образования'] = data['Уровень образования. Цифры в середине кода ОКСО хх.ХХ.хх говорят об образовательном уровне: 03 и 05 - высшее образование, 01 и 02 - среднее профессиональное образование'].apply(data_education_lvl_processor)
data_preprocessed['Описание профессии для 8-9 класса'] = data['Описание профессии для 8-9 класса'].apply(dat_description_processor)
data_preprocessed =data_preprocessed.drop(
    [
        'Специальности ОКСО - по одной специальности может быть множетсво программ обучения, которые открывают колледжии и вузы. Специальность - то, что написано у тебя в дипломе после окончания одной из программ обучения.',
        'Уровень образования. Цифры в середине кода ОКСО хх.ХХ.хх говорят об образовательном уровне: 03 и 05 - высшее образование, 01 и 02 - среднее профессиональное образование',
    ],
    axis=1
)
data_preprocessed.head(3)

,Отрасль текстом,Название профессии,Описание профессии для 8-9 класса,Специальности ОКСО,Уровень образования
№,,,,,
1,Агропромышленность,Агроном,"{'main_task': 'Выращивает ягоды, овощи, фрукты...","{'35.02.05': 'Агрономия', '35.03.03': 'Агрохим...","[SecondaryEducation, HeigherEducation, Heigher..."
2,"Архитектура, строительство и недвижимость",Маркшейдер,{'main_task': 'Исследует месторождения полезны...,"{'05.03.01': 'Геология', '21.05.04': 'Горное д...","[HeigherEducation, HeigherEducation, HeigherEd..."
3,"Архитектура, строительство и недвижимость",Реставратор,{'main_task': 'Восстанавливает старинные здани...,"{'07.02.01': 'Архитектура', '54.02.04': 'Реста...","[SecondaryEducation, SecondaryEducation, Secon..."


# Specials processing

In [111]:
specials_texts = pd.Series(sum(data_preprocessed['Специальности ОКСО'].apply(lambda a: list(a.values())).to_list(), []))
specials_nums = pd.Series(sum(data_preprocessed['Специальности ОКСО'].apply(lambda a: list(a.keys())).to_list(), []))
specials_df = pd.DataFrame({'num': specials_nums.unique(), 'name': specials_texts.unique()})
specials_texts.unique().shape, specials_texts.shape

((160,), (290,))

# Descriptions processing

In [113]:
descriptions_df = pd.DataFrame()
descriptions_df['main_task'] = data_preprocessed['Описание профессии для 8-9 класса'].apply(lambda a: a['main_task'])
descriptions_df['sub_tasks'] = data_preprocessed['Описание профессии для 8-9 класса'].apply(lambda a: a['sub_tasks'])
descriptions_df['description'] = data_preprocessed['Описание профессии для 8-9 класса'].apply(lambda a: a['description'])
descriptions_df.head()

,main_task,sub_tasks,description
№,,,
1,"Выращивает ягоды, овощи, фрукты, грибы, кормов...","[ Проверяет почву и оценивает качество семян.,...","Представь, что агроном работает на ферме, кото..."
2,"Исследует месторождения полезных ископаемых, п...",[ Исследует горную местность и определяет нали...,"Допустим, в горах строят тоннель для новой жел..."
3,"Восстанавливает старинные здания, сооружения и...",[ Изучает и анализирует старинные произведения...,"Допустим, в городе есть старинное здание. Рест..."
4,Ремонтирует строительную технику,"[ Ремонтирует строительные машины., Проводит ...","Допустим, на стройплощадке у экскаватора слома..."
5,Устанавливает и обслуживает системы водоснабже...,[ Соединяет трубы между собой — так он создает...,"Допустим, в новом здании нужно установить сист..."


In [124]:
splited_df = pd.concat([data_preprocessed, descriptions_df], axis=1)
splited_df.head()


,Отрасль текстом,Название профессии,Описание профессии для 8-9 класса,Специальности ОКСО,Уровень образования,main_task,sub_tasks,description
№,,,,,,,,
1,Агропромышленность,Агроном,"{'main_task': 'Выращивает ягоды, овощи, фрукты...","{'35.02.05': 'Агрономия', '35.03.03': 'Агрохим...","[SecondaryEducation, HeigherEducation, Heigher...","Выращивает ягоды, овощи, фрукты, грибы, кормов...","[ Проверяет почву и оценивает качество семян.,...","Представь, что агроном работает на ферме, кото..."
2,"Архитектура, строительство и недвижимость",Маркшейдер,{'main_task': 'Исследует месторождения полезны...,"{'05.03.01': 'Геология', '21.05.04': 'Горное д...","[HeigherEducation, HeigherEducation, HeigherEd...","Исследует месторождения полезных ископаемых, п...",[ Исследует горную местность и определяет нали...,"Допустим, в горах строят тоннель для новой жел..."
3,"Архитектура, строительство и недвижимость",Реставратор,{'main_task': 'Восстанавливает старинные здани...,"{'07.02.01': 'Архитектура', '54.02.04': 'Реста...","[SecondaryEducation, SecondaryEducation, Secon...","Восстанавливает старинные здания, сооружения и...",[ Изучает и анализирует старинные произведения...,"Допустим, в городе есть старинное здание. Рест..."
4,"Архитектура, строительство и недвижимость",Слесарь по ремонту строительных машин,{'main_task': 'Ремонтирует строительную техник...,{'23.02.03': 'Техническое обслуживание и ремон...,"[SecondaryEducation, SecondaryEducation, Secon...",Ремонтирует строительную технику,"[ Ремонтирует строительные машины., Проводит ...","Допустим, на стройплощадке у экскаватора слома..."
5,"Архитектура, строительство и недвижимость",Монтажник санитарно-технических систем и обору...,{'main_task': 'Устанавливает и обслуживает сис...,{'08.01.26': 'Мастер по ремонту и обслуживанию...,"[SecondaryEducation, SecondaryEducation, Secon...",Устанавливает и обслуживает системы водоснабже...,[ Соединяет трубы между собой — так он создает...,"Допустим, в новом здании нужно установить сист..."


In [162]:
def clear_from_stopwords(text, sw):
    text = text
    tokens = list(tokenize(text))
    t = []
    for i in tokens:
        if len(i.text) > 2:
            if i.text not in sw:
                t.append(i.text)

    return ' '.join(t)

In [167]:
def clear_from_stopwords_bin(text, sw):
    text = text
    tokens = list(tokenize(text))
    t = []
    for i in tokens:
        t.append(int(len(i.text) > 2 and i.text not in sw))
    return t

In [166]:
cleared_desciptions = splited_df['description'].apply(lambda a: a.lower()).apply(lambda a: clear_from_stopwords(a, sw=stop_words)).apply(lambda a: len(a.split()))
cleared_desciptions.max(), cleared_desciptions.mean(), cleared_desciptions.median(), cleared_desciptions.min()

(65, 42.16, 41.0, 15)

In [153]:
splited_df['description'].apply(lambda a: a.lower()).apply(lambda a: clear_from_stopwords_bin(a, sw=stop_words))

tensor([0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
        0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
        1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0])

In [171]:
enc = tokenizer(splited_df['description'].apply(lambda a: a.lower()).apply(lambda a: clear_from_stopwords(a, sw=stop_words)).to_list(), padding=True, truncation=True, pad_to_multiple_of=65, max_length=65, return_tensors='pt')

In [173]:
vec_desc = vectorizer(**enc)[0]

In [175]:
vec_desc.shape

torch.Size([50, 65, 1024])

In [176]:
torch.save(vec_desc, '../DATA/small_description_vectors.pt')

In [ ]:
check_text = tokenizer(['Программировать на Python легко'], padding=True, truncation=True, pad_to_multiple_of=65, max_length=65, return_tensors='pt')